In [2]:
import os
import swat
import getpass
from sasctl import Session
from sasctl.tasks import register_model, publish_model

In [4]:
password = getpass.getpass ('Enter your password : ')

In [5]:
os.environ["CAS_CLIENT_SSL_CA_LIST"] = r"C:\Users\sbxmip\.vscode\trustedcerts.pem"

In [6]:
conn = swat.CAS('https://viya4-s2.zeus.sashq-d.openstack.sas.com/cas-shared-default-http', username='sbxmip',password=password)

In [7]:
swat.cas.table.CASTable
EU_ETS_tbl = conn.CASTable('EU_ETS_CURATED', caslib='EBA')

In [ ]:
EU_ETS_tbl.columnInfo()

In [ ]:
EU_ETS_tbl.head(5)

In [ ]:
conn.loadActionSet('regression')

In [ ]:
conn.regression.glm( 
    table = dict(name = EU_ETS_tbl, where = 'year_comp = 2021'), 
    classVars=['Country_id','nace_desc','activity_desc'],
    model={'depVar':'verified',
                   'effects':['Country_id', 'activity_desc', 'nace_desc']
                  },
    store = dict(name='reg_model', replace=True),
    selection = {"method":"BACKWARD"}
)

In [25]:
astore = conn.CASTable('reg_model')

In [26]:
hostname = "https://viya4-s2.zeus.sashq-d.openstack.sas.com/"

In [ ]:
Session(hostname, 'sbxmip', password)

In [ ]:
model = register_model(astore, 'Glm_2021_back', 'Digit Project', force=True)

In [29]:
# Publish the model to a real-time scoring engine
module = publish_model(model, 'maslocal')

In [ ]:
print(module)

In [31]:
# Pass a row of data to MAS and receive the predicted result.
first_row = EU_ETS_tbl.head(1)
result = module.score(first_row)

In [ ]:
print(result)